In [5]:
import datetime
from datetime import timedelta

# get today's date
dateToday = datetime.datetime.today()

daysOfPull = 30

# minus 20 days from the date
startDate = dateToday - timedelta(days=daysOfPull)

print(startDate)

2022-12-01 12:25:44.016565


In [62]:

# Get data from NEWS API

# %pip install newsapi-python

from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='913098f1a0fd429a88cee464f86f2201')


# /v2/everything
data = []
for i in range(20):
    dateToUse = startDate + timedelta(days=i)
    date = dateToUse.strftime("%Y-%m-%d")
    all_articles = newsapi.get_everything(sources='breitbart-news,fox-news,msnbc,reuters,the-huffington-post,the-verve,bbc',from_param=date,to=date,language='en',sort_by='relevancy',page=5)
    data.append(all_articles['articles'])


In [6]:
print(data[0][0]['source']['name'])

NameError: name 'data' is not defined

In [8]:
convertedData = []
for i in range(len(data)):
    for j in range(len(data[i])):
        convertedData.append({"sentence": data[i][j]['description'], "pub_year": data[i][j]['publishedAt'][0:4], "news_link": data[i][j]['url'], "outlet" : data[i][j]['source']['name']})
        # convertedData.append(data[i]['articles'][0])

print(len(convertedData))

# use pandas to create dataframe
import pandas as pd

NameError: name 'data' is not defined

In [7]:
import numpy as np

print(convertedData[200])

NameError: name 'convertedData' is not defined

In [70]:
# add converted data to new dataframe
df = pd.DataFrame()
df = pd.DataFrame(convertedData)
df.head()

,sentence,pub_year,news_link,outlet
0,"Surveillance video shows Austin, Texas, attorn...",2022,https://www.breitbart.com/politics/2022/12/01/...,Breitbart News
1,Rapper and entrepreneur Kanye West praised Hit...,2022,https://www.breitbart.com/politics/2022/12/01/...,Breitbart News
2,MSNBC political commentator and former White H...,2022,https://www.breitbart.com/clips/2022/12/01/psa...,Breitbart News
3,Elon Musk's brain chip company Neuralink aims ...,2022,https://www.breitbart.com/tech/2022/12/01/elon...,Breitbart News
4,Rep. Michael Waltz (R) discusses the lockdown ...,2022,https://www.breitbart.com/podcast/2022/12/01/b...,Breitbart News


In [74]:
# Save to csv
path = 'api_data_'+ str(len(convertedData)) + '.csv'
df.to_csv(path, index=False)

In [5]:
# import saved prediction model from local file
%pip install tensorflow==2.10.0 --user

import tensorflow as tf
# show version of tensorflow
print(tf.__version__)
# show version of keras
print(tf.keras.__version__)


  Using cached tensorflow-2.10.0-cp310-cp310-win_amd64.whl (455.9 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
Note: you may need to restart the kernel to use updated packages.
2.9.0
2.9.0


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.8.2 requires tensorflow<2.9,>=2.8.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.10.0 which is incompatible.
tensorflow-intel 2.11.0 requires keras<2.12,>=2.11.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.11.0 requires tensorboard<2.12,>=2.11, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.11.0 requires tensorflow-estimator<2.12,>=2.11.0, but you have tensorflow-estimator 2.10.0 which is incompatible.


In [2]:

import tensorflow as tf
print(tf.__version__)
# show version of keras
print(tf.keras.__version__)

2.10.0
2.10.0


In [3]:
model = tf.keras.models.load_model('C://users/verni/Desktop/AI course/saved_models/bias_model/')

In [16]:
#import model
from transformers import BertTokenizer
import pandas as pd
import numpy as np

# Load model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def tokenize(sentences, max_length=512, padding='max_length'):

    return tokenizer(
        sentences,
        truncation=True,
        padding=padding,
        max_length=max_length,
        return_tensors="tf"
    )

# Load data
df = pd.read_csv('api_data_1400.csv')

#add a column for predictions
df['pred'] = ""
df.head()

# Tokenize data

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
# for each item in the dataframe
for i in range(len(df)):
    # tokenize the sentence
    inputs = dict(tokenizer([df.sentence[i]], return_tensors='tf'))
    prediction = model.predict(inputs)
    df.pred[i] = tf.nn.softmax(prediction['logits'], axis=1).numpy()


1/1 [==============================] - 0s 128ms/step


C:\Users\verni\AppData\Local\Temp\ipykernel_7208\626229674.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.pred[i] = tf.nn.softmax(prediction['logits'], axis=1).numpy()


1/1 [==============================] - 0s 139ms/step


In [17]:
print(df.head())

                                            sentence  pub_year  \
0  Surveillance video shows Austin, Texas, attorn...      2022   
1  Rapper and entrepreneur Kanye West praised Hit...      2022   
2  MSNBC political commentator and former White H...      2022   
3  Elon Musk's brain chip company Neuralink aims ...      2022   
4  Rep. Michael Waltz (R) discusses the lockdown ...      2022   

                                           news_link          outlet  \
0  https://www.breitbart.com/politics/2022/12/01/...  Breitbart News   
1  https://www.breitbart.com/politics/2022/12/01/...  Breitbart News   
2  https://www.breitbart.com/clips/2022/12/01/psa...  Breitbart News   
3  https://www.breitbart.com/tech/2022/12/01/elon...  Breitbart News   
4  https://www.breitbart.com/podcast/2022/12/01/b...  Breitbart News   

                        pred  
0  [[0.8682157, 0.13178426]]  
1   [[0.3139803, 0.6860197]]  
2   [[0.4305216, 0.5694784]]  
3  [[0.9005478, 0.09945217]]  
4  [[0.8563726,

In [20]:
# consider pred and add label to dataframe
df['label'] = ""
for i in range(len(df)):
    if df.pred[i][0][0] > df.pred[i][0][1]:
        if df.pred[i][0][0] > 0.8:
            df.label[i] = 0
    else:
        if df.pred[i][0][1] > 0.8:
            df.label[i] = 1

C:\Users\verni\AppData\Local\Temp\ipykernel_7208\91864051.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.label[i] = 0
C:\Users\verni\AppData\Local\Temp\ipykernel_7208\91864051.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.label[i] = 1


In [21]:
# remove entries where label is empty
df = df[df.label != '']

print(df.head())

                                            sentence  pub_year  \
0  Surveillance video shows Austin, Texas, attorn...      2022   
3  Elon Musk's brain chip company Neuralink aims ...      2022   
4  Rep. Michael Waltz (R) discusses the lockdown ...      2022   
5  A New York man who brutally attacked a 67-year...      2022   
7  The Joe Biden administration reportedly placed...      2022   

                                           news_link          outlet  \
0  https://www.breitbart.com/politics/2022/12/01/...  Breitbart News   
3  https://www.breitbart.com/tech/2022/12/01/elon...  Breitbart News   
4  https://www.breitbart.com/podcast/2022/12/01/b...  Breitbart News   
5  https://www.breitbart.com/crime/2022/11/30/new...  Breitbart News   
7  https://www.breitbart.com/politics/2022/11/30/...  Breitbart News   

                         pred label  
0   [[0.8682157, 0.13178426]]     0  
3   [[0.9005478, 0.09945217]]     0  
4   [[0.8563726, 0.14362742]]     0  
5  [[0.92137825, 0

In [22]:
# if label is 0 or 1, add to csv file
print(len(df))
df.to_csv('pseudo_labeled_data.csv', index=False)


1079
